### Метрики энкодера на основе Resnet18.  
#### Выходной слой: *nn.Linear(in_features=512, out_features=1024, bias=True)*

### Визуализация в 3 ГК помимо того что не дает колличественных оценок точности энкодера, так и несет в себе в лучшем случае около 40% информации от выходного вектора длинной 1024. 

###  Необходимо ознакомится с метриками и оценками модели энкодера. исп.:
* kMeans
* OneClass SVM
* Gaussian Mixture

### Конечная цель: оценка целесообразности применения энкодера в рамках *данной* задачи.

Что откуда качать:

* https://drive.google.com/file/d/1-oIPyg3uFT1n--MXyR4Uzx95YqR3NsNT/view?usp=sharing - дополнительные знаки - не референсные. Часть из них - вырезка из видосов, часть - собранно ручками. Разместить в папке *data/additional_sign/*
* https://drive.google.com/file/d/1-rTwhmdUdcuPMYz8BiPQV3fiWCSJjE20/view?usp=sharing - *last_encoder_1024_98* - веса энкодера. Разместить в папке с ноутбуком.
* https://drive.google.com/file/d/1-K3ee1NbMmx_0T5uwMesStmKnZO_6mWi/view?usp=sharing - rtds с csv, содержащей инфу. Разместить в папке *data*: data/R_MERGED/.. и data/RTDS_DATASET.csv.
* https://drive.google.com/file/d/1-l3VvU-WtSoXbW_AaTFUreVD-tgXV8Q0/view?usp=sharing - стоковые знаки. Используются как референс, то есть объеденяются с rtds с пометкой 'train'. Разместить так: data/STOCK_SIGNS.

In [ ]:
import albumentations as A
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
from torch import nn
import seaborn as sns
import pandas as pd
import os
import pathlib
import shutil
import cv2
import PIL
import cv2
import sys
from datetime import datetime

TEXT_COLOR = 'black'
# Зафиксируем состояние случайных чисел
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
random.seed(RANDOM_STATE)
%matplotlib inline
plt.rcParams["figure.figsize"] = (17,10)

USE_COLAB_GPU = False
IN_COLAB = False

try:
    import google.colab
    IN_COLAB = True
    USE_COLAB_GPU = True
    from google.colab import drive
except:
    if IN_COLAB:
        print('[!] YOU ARE IN COLAB, BUT DIDNT MOUND A DRIVE. Model wont be synced[!]')

        if not os.path.isfile(CURRENT_FILE_NAME):
            print("FIX ME")
        IN_COLAB = False

    else:
        print('[!] RUNNING NOT IN COLAB')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
if not IN_COLAB:
    PROJECT_ROOT = pathlib.Path(os.path.join(os.curdir, os.pardir))
else:
    PROJECT_ROOT = pathlib.Path('..')
    
DATA_DIR = PROJECT_ROOT / 'data'
NOTEBOOKS_DIR = PROJECT_ROOT / 'notebooks'
SRC_PATH = str(PROJECT_ROOT / 'src')

if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)
    
torch.set_grad_enabled(False)

In [ ]:
from torchvision.models import resnet
encoder = resnet.resnet18(pretrained=True)
encoder.fc = nn.Linear(in_features=512, out_features=512, bias=True)
r = encoder.load_state_dict(torch.load('last_encoder403_24')['model'])
encoder.eval()
encoder.to(device)
assert r

In [ ]:
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader

def getDataLoaderFromDataset(dataset, shuffle=False, drop_last=True):
    
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,
        shuffle=shuffle,
        drop_last=drop_last
    )
    
    return loader


@torch.no_grad()
def simpleGetAllEmbeddings(model, dataset, batch_size, dsc=''):
    
    dataloader = getDataLoaderFromDataset(
        dataset,
        shuffle=True,
        drop_last=False
    )
    
    s, e = 0, 0
    pbar = tqdm(
        enumerate(dataloader), 
        total=len(dataloader),
        position=0,
        leave=False,
        desc='Getting all embeddings...' + dsc)
    info_arr = []
    
    add_info_len = None
    
    for idx, (data, labels, info) in pbar:
        data = data.to(device)
        
        q = model(data)
        
        if labels.dim() == 1:
            labels = labels.unsqueeze(1)
        if idx == 0:
            labels_ret = torch.zeros(
                len(dataloader.dataset),
                labels.size(1),
                device=device,
                dtype=labels.dtype,
            )
            all_q = torch.zeros(
                len(dataloader.dataset),
                q.size(1),
                device=device,
                dtype=q.dtype,
            )
        
        info = np.array(info)
        if add_info_len == None:
            add_info_len = info.shape[0]
        
        info_arr.extend(info.T.reshape((-1, add_info_len)))
        e = s + q.size(0)
        all_q[s:e] = q
        labels_ret[s:e] = labels
        s = e  
    
    all_q = torch.nn.functional.normalize(all_q)
    return all_q, labels_ret, info_arr

### Этап 1.1. Берем RTDS, из него берем *train* как *baseline*. Заменяем *valid* на *test*.

In [ ]:
RTDS_DF = pd.read_csv(DATA_DIR / 'RTDS_DATASET.csv')
RTDS_DF['filepath'] = RTDS_DF['filepath'].apply(lambda x: str(DATA_DIR / x))
RTDS_DF.drop_duplicates(subset=['filepath'], inplace=True)
RTDS_DF['SET'] = RTDS_DF['SET'].apply(lambda x: 'test' if x == 'valid' else x)

# print(set(RTDS_DF['SET']))
MAKE_ONCE_MERGE_FLAG = True
RTDS_DF

### *train* как референс, *valid* - query для валидации.
### Этап 1.2. Формируем DataFrame отсутствущих знаков в RTDS.

#### Напоминание чего не хватает. Обр. внимание на 3.25. Там для каждоый скорости потенциально.

| Знак | Описание | Источник |
| ------------- | ------------- | ---- |
| 1.6 | Пересечение равнозначных дорог | - |
| 1.31 | Туннель | - |
| 2.4 | Уступите дорогу | GTSRB Recognition |
| 3.21 | Конец запрещения обгона | GTSRB Recognition |
| 3.22 | Обгон грузовым автомобилям запрещен | GTSRB Recognition |
| 3.23 | Конец запрещения обгона грузовым автомобилям | GTSRB Recognition |
| 3.24-90 | Огр 90 | - |
| 3.24-100 | Огр 100 | GTSRB Recognition |
| 3.24-110 | Огр 110 | - |
| 3.24-120 | Огр 120 | GTSRB Recognition |
| 3.24-130 | Огр 130 | - |
| **3.25** | **Конец огр. максимальной скорости** | **GTSRB Recognition** |
| 3.31 | Конец всех ограничений | GTSRB Recognition |
| 6.3.2 | Зона для разворота | - |

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

additional_DF = pd.DataFrame(columns=RTDS_DF.columns)

encode_offset = max(set(RTDS_DF['ENCODED_LABEL'])) + 1
files = os.listdir(DATA_DIR / 'additional_sign')

FLAG_FIG_3_25 = True

for file in files:
    sign = file.split('_')[0]
    
    if FLAG_FIG_3_25 and sign.rsplit('.', 1)[0] == '3.25':
        sign = '3.25'
        
    if FLAG_FIG_3_25 and '3.24' in sign.rsplit('.', 1)[0]:
        continue
        sign = '3.24'         

    row = {'filepath': str(DATA_DIR / 'additional_sign' / file), 
           'SIGN': sign, 
           'SET': 'test'
          } 
    additional_DF = additional_DF.append(row, ignore_index=True)

additional_DF.ENCODED_LABEL = le.fit_transform(additional_DF.SIGN) + len(set(RTDS_DF.ENCODED_LABEL))
# display(additional_DF)

if MAKE_ONCE_MERGE_FLAG:
    RTDS_DF = RTDS_DF.append(additional_DF, ignore_index=True)
    MAKE_ONCE_FLAG = False

additional_DF.groupby(['SIGN', 'ENCODED_LABEL']).agg(['count'])

In [ ]:
len(set(RTDS_DF.SIGN))

In [ ]:
len(set(RTDS_DF.ENCODED_LABEL))

In [ ]:
LABEL_DICT = dict(zip(RTDS_DF.SIGN, RTDS_DF.ENCODED_LABEL))
# LABEL_DICT

In [ ]:
len(LABEL_DICT)

### Этап 2. Формируем для отсутствующих~=**ДЛЯ ВСЕХ** знаков baseline из образцовых знаков с википедии.

In [ ]:
STOCK_SIGNS_CSV_LOCATION = DATA_DIR / 'STOCK_SIGNS.csv'
STOCK_SIGNS_DATAFRAME = pd.read_csv(STOCK_SIGNS_CSV_LOCATION)

STOCK_SIGNS_DATAFRAME.loc[STOCK_SIGNS_DATAFRAME['SIGN'] == '5.19.2', 'SIGN'] = '5.19.1'
if FLAG_FIG_3_25:
    STOCK_SIGNS_DATAFRAME['SIGN'] = STOCK_SIGNS_DATAFRAME['SIGN'].apply(
        lambda x: '3.25' if x.rsplit('.', 1)[0] == '3.25' else x)

## FIXUP для проблем описанных ниже
STOCK_SIGNS_DATAFRAME['SIGN'] = STOCK_SIGNS_DATAFRAME['SIGN'].apply(
        lambda x: '3.18' if x.rsplit('.', 1)[0] == '3.18' else x)

STOCK_SIGNS_DATAFRAME['SIGN'] = STOCK_SIGNS_DATAFRAME['SIGN'].apply(
        lambda x: '2.3' if x.rsplit('.', 1)[0] == '2.3' else x)

STOCK_SIGNS_DATAFRAME['SIGN'] = STOCK_SIGNS_DATAFRAME['SIGN'].apply(
        lambda x: '3.24' if '3.24' in x.rsplit('.', 1)[0] else x)

STOCK_SIGNS_DATAFRAME['filepath'] = STOCK_SIGNS_DATAFRAME['filepath'].apply(lambda x: str(DATA_DIR / x))
STOCK_SIGNS_DATAFRAME['ENCODED_LABEL'] = [LABEL_DICT[i] for i in STOCK_SIGNS_DATAFRAME['SIGN']]

STOCK_SIGNS_DATAFRAME['SET'] = 'train'

RTDS_DF = RTDS_DF.append(STOCK_SIGNS_DATAFRAME, ignore_index=True).reset_index()
    
STOCK_SIGNS_DATAFRAME[::6]

In [ ]:
len(set(STOCK_SIGNS_DATAFRAME['ENCODED_LABEL']))

In [ ]:
set(RTDS_DF['SET'])

## БЕДА, RTDS объеденяет все 3.18 в одну группу. Еще проблемные: 2.3.1. Ну пофиг блин) Смотрим на ошибки.

### Baseline готов, тестовый датасет готов. Че хотим? Хотим получить какие-нибудь метрики.

In [ ]:
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.augmentations.transforms import PadIfNeeded
from albumentations.augmentations.geometric.resize import LongestMaxSize

img_size = 40

minimal_transform = A.Compose(
        [
        LongestMaxSize(
            img_size,
            interpolation=cv2.INTER_AREA  
        ),
        PadIfNeeded(
            img_size, 
            img_size, 
            border_mode=cv2.BORDER_CONSTANT, 
            value=0
        ),
        ToTensorV2(),
        ]
    )

class SignDataset(torch.utils.data.Dataset):
    def __init__(self, df, set_label=None, hyp=None, transform=None, alpha_color=None):
                
        self.transform = transform
        
        if set_label == None:
            self.df = df
        else:
            self.df = df[df['SET']==set_label]
        
        self.hyp = hyp
        self.alpha_color = alpha_color
    def __len__(self):
        return len(self.df.index)
    
    def __getitem__(self, index): 
        label = int(self.df.iloc[index]['ENCODED_LABEL'])
        path = str(self.df.iloc[index]['filepath'])
        sign = str(self.df.iloc[index]['SIGN'])
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        
        # check does it contains transparent channel 
        if img.shape[2] == 4:
        # randomize transparent
            trans_mask = img[:,:,3] == 0
            img[trans_mask] = [self.alpha_color if self.alpha_color else random.randrange(0, 256), 
                               self.alpha_color if self.alpha_color else random.randrange(0, 256), 
                               self.alpha_color if self.alpha_color else random.randrange(0, 256), 
                               255]

            img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        # /randomize transparent
                
        # augment 
        if self.transform:
            img = self.transform(image=img)['image']
        # /augment
        
        img = img / 255
        return img, label, (path, sign)

train_dataset = SignDataset(RTDS_DF, 
                            set_label='train', 
                            transform=minimal_transform, 
                            hyp=None,
                            alpha_color=144
                           )

valid_dataset = SignDataset(RTDS_DF, 
                            set_label='test',  
                            transform=minimal_transform, 
                            hyp=None,
                            alpha_color=144
                           )

nrows, ncols = 70, 6
fig = plt.figure(figsize = (16,200))


for idx, (img, encoded_label, (path, sign)) in enumerate(train_dataset):
    
    if idx > 5:
        break
    img = torch.Tensor.permute(img, [1, 2, 0]).numpy() 
    ax = fig.add_subplot(nrows, ncols, idx+1)
        
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), aspect=1)
    ax.set_title(str(sign), fontsize=15)
    
plt.tight_layout()

In [ ]:
batch_size = 96
num_workers = 2 if IN_COLAB else 0

encoder.eval()
train_embeddings, train_labels, train_additional_info = simpleGetAllEmbeddings(
    encoder, train_dataset, batch_size, ' for train'
)

test_embeddings, test_labels, test_additional_info = simpleGetAllEmbeddings(
    encoder, valid_dataset, batch_size, ' for test'
)

train_labels = train_labels.squeeze(1)
test_labels = test_labels.squeeze(1)

In [ ]:
print(test_labels.unique())

In [ ]:
print(train_labels.unique())

In [ ]:
RTDS_DF.groupby(['SET']).agg('count')

### Выше все ок, тестовый набор содержит тест+валид, который сформирован ноутбуков *RTSD-R_MERGED.ipynb*. В валид попало много знаков пешеходного перехода, т.к. их количество значительно превосходило остальные.

## Get Centroids

In [ ]:
INVERSED_LABEL_DICT = {v: k for k, v in LABEL_DICT.items()}

In [ ]:
labels_list = train_labels.cpu().numpy()
labels_set = list(set(labels_list))

embeddingsListForCentroids = train_embeddings.cpu().numpy()
centroidLocationDict = {}

for idx, label in enumerate(labels_set):
    print('Current label:', label, '[%s]' % INVERSED_LABEL_DICT[label])
    mask = labels_list == label
    
    currentLabelEmbeddingsForCentroids = embeddingsListForCentroids[mask]
    zipped = list(zip(*currentLabelEmbeddingsForCentroids))
    
    singleCoord = []
    for coord in zipped:
        coord = sum(coord) / len(coord)
        singleCoord.append(coord)
        # print(coord)
        
    centroidLocationDict[label] = singleCoord

### lets construct plot DataFrame
Фундаментальный вопрос. fit PCA надо делать на train или на train+test?

In [ ]:
from sklearn.decomposition import PCA
dim3 = True

coords = ['x', 'y'] + (['z'] if dim3 else [])
columns = [*coords, 'type', 'size', 'sign', 'filepath', 'color', 'marker']

plot_df = pd.DataFrame(columns=columns)
# display(plot_df)

reducer = PCA(
    n_components=3 if dim3 else 2, 
    random_state=RANDOM_STATE
)

train_size = 2
train_type = 'train'


test_size = 2
test_type = 'test'


centroid_size = 10
centroid_type = 'centroid'


## FIT REDUCER
train_embeddings_ = reducer.fit_transform(train_embeddings.cpu().numpy())

from itertools import cycle
import plotly.express as px
palette = cycle(
        [*px.colors.qualitative.Dark24, 
         *px.colors.qualitative.Alphabet, 
         *px.colors.qualitative.Light24]
    )
colorDict = {}

listOfRows = []
## CENTROIDS
for k, v in centroidLocationDict.items():
    coords = reducer.transform(np.array(v).reshape(1, -1)).flatten()
    # print([*coords, centroid_type, centroid_size, INVERSED_LABEL_DICT[k]])
    path = STOCK_SIGNS_DATAFRAME[STOCK_SIGNS_DATAFRAME['SIGN'] == INVERSED_LABEL_DICT[k]]['filepath'].values[0]
    
    colorDict[INVERSED_LABEL_DICT[k]] = next(palette)
    
    row = pd.Series(
        [*coords, centroid_type, centroid_size, INVERSED_LABEL_DICT[k], path, 
         colorDict[INVERSED_LABEL_DICT[k]], 'diamond'],
        index=plot_df.columns,
    )
    listOfRows.append(row)
    
## TRAIN
for idx, (coord, label, info) in tqdm(
    enumerate(
        zip(train_embeddings_, train_labels, train_additional_info)),
    total=len(train_labels)
):
    label = label.cpu().numpy()
    
    color = colorDict[INVERSED_LABEL_DICT[int(label)]]
    
    row = pd.Series(
        [*coord, train_type, train_size, INVERSED_LABEL_DICT[int(label)], info[0], 
         colorDict[INVERSED_LABEL_DICT[k]], 'circle'],
        index=plot_df.columns,
    )
    listOfRows.append(row)


    
## TEST
test_embeddings_ = reducer.transform(test_embeddings.cpu().numpy())

for idx, (coord, label, info) in tqdm(
    enumerate(
        zip(test_embeddings_, test_labels, test_additional_info)),
    total=len(test_labels)
):
    label = label.cpu().numpy()
    
    row = pd.Series(
        [*coord, test_type, test_size, info[1], info[0], 
        colorDict[INVERSED_LABEL_DICT[k]], 'circle'],
        index=plot_df.columns,
    )
    
    listOfRows.append(row)

plot_df = plot_df.append(listOfRows)
plot_df['x'] = plot_df['x'].astype(float)
plot_df['y'] = plot_df['y'].astype(float)
if 'z' in plot_df.columns:
    plot_df['z'] = plot_df['z'].astype(float)
    
plot_df['size'] = plot_df['size'].astype(int)

In [ ]:
sum(reducer.explained_variance_ratio_)

In [ ]:
import plotly.express as px
from plotly import graph_objects as go 
from itertools import cycle

## WERKZEUG FIX
import logging
log = logging.getLogger('werkzeug')
log.setLevel(logging.ERROR)
## /WERKZEUG FIX

PLOT_CENTROID_AND_TEST_ONLY = True
PLOT_LIMIT_FRAC = 0.6

if PLOT_CENTROID_AND_TEST_ONLY:
    plot_df_ = plot_df[plot_df['type'] != 'train'][::-1]
else:
    plot_df_ = plot_df[::-1]

if PLOT_LIMIT_FRAC:
    plot_df_ = plot_df_.groupby(['sign', 'type']).sample(frac=PLOT_LIMIT_FRAC)

from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output, no_update
import base64


app = JupyterDash(__name__)

@app.callback(
    Output("graph-tooltip-5", "show"),
    Output("graph-tooltip-5", "bbox"),
    Output("graph-tooltip-5", "children"),
    Input("graph-5", "hoverData"),
)
def display_hover(hoverData):

    # print(hoverData)
    # return False, no_update, no_update
    if hoverData is None:
        return False, no_update, no_update

    hover_data = hoverData["points"][0]
    bbox = hover_data["bbox"]
    num = hover_data["pointNumber"]
    data = hover_data['customdata']
    sign = data[0]
    rel_img_path = data[1]
    
    try:
        with open(rel_img_path, 'rb') as f:
            image = f.read()
    except:
        print(data)
        return False, no_update, no_update
    
    b64sed_image = 'data:image/png;base64,' + base64.b64encode(image).decode('utf-8')

    children = [
        html.Div([
            html.Img(
                src=b64sed_image,
                style={"width": "70px", 'display': 'block', 'margin': '0 auto'},
            ),
            html.P(sign, style={"fontSize": 14, 'text-align':'center'}),
            html.P(rel_img_path, style={"fontSize": 10}),
        ])
    ]
    return True, bbox, children



dim3 = True
plot_args = {
    'x': 'x',
    'y': 'y',
    'color': 'sign',
    'size': 'size',
    'opacity': 0.2 if dim3 else 0.5,
    'symbol': 'type',
    'hover_name': 'sign',
    'hover_data': ['sign', 'filepath', 'type'],
    'animation_group': 'type',
    'color_discrete_sequence': [
        *px.colors.qualitative.Dark24, 
        # *px.colors.qualitative.Alphabet, 
        # *px.colors.qualitative.Light24
    ]
}

if dim3:
    plotFcn = px.scatter_3d
    plot_args.update({'z': 'z'})
else:
    plotFcn = px.scatter
      

fig = plotFcn(
        plot_df_,
        **plot_args,

    )

fig.update_traces(
        hoverinfo="none", 
        hovertemplate=None,
        marker=dict(
            line=dict(
                width=0)
           )
)
    
fig.update_layout(
        width=950,
        height=950)

## FIX ZORDER
if True:
    sampleData = list(fig.data)
    centroidsList = []
    for t in list(sampleData[:]):
        if (t.ids[0] == 'centroid'):
            temp_t = t
            sampleData.remove(t)
            temp_t['marker']['opacity'] = 1
            temp_t['text'] = temp_t['customdata'][0][0]
            temp_t['textposition'] = 'top center'
            temp_t['mode'] = 'markers+text'
            temp_t['marker']['line']['width'] = 40 if dim3 else 2
            temp_t['marker']['line']['color'] = 'rgb(0, 0, 0)'
            centroidsList.append(temp_t)
            
    fig.data = tuple(sampleData + centroidsList)
    
fig.update_layout(font=dict(size=18))

app.layout = html.Div(
            className="container",
            children=[
                dcc.Graph(id="graph-5", figure=fig, clear_on_unhover=True),
                dcc.Tooltip(id="graph-tooltip-5", direction='bottom'),
            ],
        )
    
#fig.show()
if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=2002)

In [ ]:
def getEulerDistance(a, b):
    squares = [(p-q) ** 2 for p, q in zip(a, b)]
    return sum(squares) ** .5
    
distancesSign = {}

centroidSignList = list(centroidLocationDict.keys())
# print(centroidSignList)
for idx, ikey in enumerate(centroidSignList):
    distancesSign[INVERSED_LABEL_DICT[ikey]] = {}
    
    
    distancesSign[INVERSED_LABEL_DICT[ikey]][INVERSED_LABEL_DICT[ikey]] = np.NaN
    for jdx, jkey in enumerate(centroidSignList[idx + 1:]):
        dist = getEulerDistance(
            centroidLocationDict[ikey],
            centroidLocationDict[jkey]
        )
        distancesSign[INVERSED_LABEL_DICT[ikey]][INVERSED_LABEL_DICT[jkey]] = dist
        # distancesSign[INVERSED_LABEL_DICT[jkey]] = {}
        # distancesSign[INVERSED_LABEL_DICT[jkey]][INVERSED_LABEL_DICT[ikey]] = dist
        # break
        
# distancesSign
df = pd.DataFrame.from_dict(distancesSign)
for i in range(len(df)):
    df.iloc[i] = df.iloc[:, i]

In [ ]:
figH = px.imshow(df)
# fig.show()

app1 = JupyterDash(__name__)

PATH_PREFIX = '../data/STOCK_SIGNS/'
PATH_POSTFIX = '.png'

@app1.callback(
    Output("graph-tooltip-5", "show"),
    Output("graph-tooltip-5", "bbox"),
    Output("graph-tooltip-5", "children"),
    Input("graph-5", "hoverData"),
)
def display_hover(hoverData):
    if hoverData is None:
        return False, no_update, no_update

    hover_data = hoverData["points"][0]
    
    hover_data['x'] = '2.3.1' if hover_data['x'] == '2.3' else hover_data['x']
    hover_data['y'] = '2.3.1' if hover_data['y'] == '2.3' else hover_data['y']        
    hover_data['x'] = '3.18.1'  if hover_data['x'] == '3.18' else hover_data['x'] 
    hover_data['y'] = '3.18.1'  if hover_data['y'] == '3.18' else hover_data['y']
        
    x_img_path = PATH_PREFIX + hover_data['x'] + PATH_POSTFIX
    y_img_path = PATH_PREFIX + hover_data['y'] + PATH_POSTFIX
    
    bbox = hover_data["bbox"]
    
    try:
        with open(x_img_path, 'rb') as f:
            image1 = f.read()
        with open(y_img_path, 'rb') as f:
            image2 = f.read()
    except:
        print(hoverData)
        return False, no_update, no_update
    
    img1 = 'data:image/png;base64,' + base64.b64encode(image1).decode('utf-8')
    img2 = 'data:image/png;base64,' + base64.b64encode(image2).decode('utf-8')

    children = [
        html.Div([
            html.Img(
                src=img1,
                style={"width": "70px",  'margin': '0 auto'},
            ),
            html.Img(
                src=img2,
                style={"width": "70px",  'margin': '0 auto'},
            ),
            html.P(hover_data['x'] + ':' + hover_data['y'], style={"fontSize": 14, 'text-align':'center'}),
            html.P(str(hover_data['z']), style={"fontSize": 14, 'text-align':'center'}),
            
        ]),
    ]
    return True, bbox, children

figH.update_traces(
        hoverinfo="none", 
        hovertemplate=None
)
    
figH.update_layout(
        width=600,
        height=600)

app1.layout = html.Div(
            className="container",
            children=[
                dcc.Graph(id="graph-5", figure=figH, clear_on_unhover=True),
                dcc.Tooltip(id="graph-tooltip-5", direction='bottom'),
            ],
        )
    
#fig.show()
if __name__ == '__main__':
    app1.run_server(mode='inline', debug=True, port=2003)

## Confusion Matrix

* train_embeddings, train_labels, train_additional_info - в этих переменных вся инфа о тестовых картинках;
* centroidLocationDict - словарь центроидов
* getEulerDistance - функция эвклидова расстояния.

In [ ]:
from tqdm.notebook import trange
centroidLocationDictGpu = {}
for key, item in centroidLocationDict.items():
    centroidLocationDictGpu[key] = torch.Tensor(item).to(device)

In [ ]:
# %%prun -s cumulative

cfDict = {}
CHECK_TRAIN = False
for i in trange(len(train_embeddings if CHECK_TRAIN else test_embeddings)):
    # 1.1 transform embediding to list
    emb = train_embeddings[i] if CHECK_TRAIN else test_embeddings[i]
    # 1.2. get closer centroid
    
    minDist = 999999
    closerVal = -1
    
    for key, item in centroidLocationDictGpu.items():
        dist = (emb - item).pow(2).sum(0).sqrt()
        #print(dist)
        # dist = getEulerDistance(emb.tolist(), item.tolist())
        #print(dist)
        if dist < minDist:
            minDist = dist
            closerVal = key
    
    realSign = INVERSED_LABEL_DICT[train_labels[i].item() if CHECK_TRAIN else test_labels[i].item()]
    predictedSign =  INVERSED_LABEL_DICT[closerVal]
    # print('for', realSign, 'predicted', predictedSign)
    
    if not realSign in cfDict:
        cfDict[realSign] = {} 
    if not predictedSign in cfDict[realSign]:
        cfDict[realSign][predictedSign] = 0
        
    cfDict[realSign][predictedSign] += 1


In [ ]:
cfDict['7.4']

In [ ]:
cfDf = pd.DataFrame.from_dict(dict(sorted(cfDict.items())))
# for i in range(len(cfDf)):
#    cfDf.iloc[i] = cfDf.iloc[:, i]
cfDf = cfDf.sort_index().T

SHOILD_I_NORMOLIZE = False
if SHOILD_I_NORMOLIZE:
    cfDf = cfDf.apply(lambda x: x / x.sum())


In [ ]:
figCf = px.imshow(cfDf.apply(lambda x: x / x.sum(), axis=1), color_continuous_scale=px.colors.sequential.Cividis_r)

app2 = JupyterDash(__name__)

PATH_PREFIX = '../data/STOCK_SIGNS/'
PATH_POSTFIX = '.png'

@app2.callback(
    Output("graph-tooltip-5", "show"),
    Output("graph-tooltip-5", "bbox"),
    Output("graph-tooltip-5", "children"),
    Input("graph-5", "hoverData"),
)
def display_hover(hoverData):
    if hoverData is None:
        return False, no_update, no_update

    hover_data = hoverData["points"][0]
    sum_x = sum(map(int, cfDict[hover_data['y']].values()))
    
    if not hover_data['z']:
        return False, no_update, no_update
    
    hover_data['x'] = '2.3.1' if hover_data['x'] == '2.3' else hover_data['x']
    hover_data['y'] = '2.3.1' if hover_data['y'] == '2.3' else hover_data['y']        
    hover_data['x'] = '3.18.1'  if hover_data['x'] == '3.18' else hover_data['x'] 
    hover_data['y'] = '3.18.1'  if hover_data['y'] == '3.18' else hover_data['y']
    hover_data['x'] = '3.25.10' if hover_data['x'] == '3.25' else hover_data['x']
    hover_data['y'] = '3.25.10' if hover_data['y'] == '3.25' else hover_data['y']         
    hover_data['x'] = '3.24.10' if hover_data['x'] == '3.24' else hover_data['x']
    hover_data['y'] = '3.24.10' if hover_data['y'] == '3.24' else hover_data['y']   
    
    x_img_path = PATH_PREFIX + hover_data['x'] + PATH_POSTFIX
    y_img_path = PATH_PREFIX + hover_data['y'] + PATH_POSTFIX
    
    bbox = hover_data["bbox"]
    
    try:
        with open(x_img_path, 'rb') as f:
            image1 = f.read()
        with open(y_img_path, 'rb') as f:
            image2 = f.read()
    except:
        # print(hoverData)
        return False, no_update, no_update
    
    img1 = 'data:image/png;base64,' + base64.b64encode(image1).decode('utf-8')
    img2 = 'data:image/png;base64,' + base64.b64encode(image2).decode('utf-8')

    children = [
        html.Div([
            html.Img(
                src=img1,
                style={"width": "70px",  'margin': '0 auto'},
            ),
            html.Img(
                src=img2,
                style={"width": "70px",  'margin': '0 auto'},
            ),
            html.P(hover_data['x'] + ':' + hover_data['y'], style={"fontSize": 14, 'text-align':'center'}),
            html.P(
                str(hover_data['z']) 
                + ': all:' + str(sum_x * hover_data['z']), 
                style={"fontSize": 14, 'text-align':'center'}
            ),
        ]),
    ]
    return True, bbox, children

figCf.update_traces(
        hoverinfo="none", 
        hovertemplate=None
)
    
figCf.update_layout(
        width=950,
        height=650)
# figCf['layout'].update(plot_bgcolor='green')
app2.layout = html.Div(
            className="container",
            children=[
                dcc.Graph(id="graph-5", figure=figCf, clear_on_unhover=True),
                dcc.Tooltip(id="graph-tooltip-5", direction='bottom'),
            ],
        )
    
#fig.show()
if __name__ == '__main__':
    app2.run_server(mode='inline', debug=True, port=2003)

Precision/F1

Значения precision находятся в матрице ниже. В строках - актульные значения, в столбцах точность/вероятность предсказывания соответсвующего знака.

In [ ]:
cfDf

In [ ]:
TPdict = {}
FNdict = {}
FPdict = {}
TNdict = {}

cfDf = cfDf.T

for i, row in cfDf.iterrows():    
    try:
        TPdict[i] = cfDf[i][i]
        FNdict[i] = cfDf[i].sum() - TPdict[i]
        FPdict[i] = cfDf.loc[i].sum() - TPdict[i]
        TNdict[i] = cfDf.fillna(0).values.sum() - TPdict[i] - FNdict[i] - FPdict[i]
    except:
        print('err for key', i)
cfDf = cfDf.T

In [ ]:
_sign = '7.4'
TPdict[_sign], FNdict[_sign], FPdict[_sign], TNdict[_sign]

In [ ]:
PrecisionDict = {}
RecallDict = {}
F1Dict = {}
SupportDict = {}
for i in TPdict.keys():
    PrecisionDict[i] = TPdict[i] / (TPdict[i] + FPdict[i])
    RecallDict[i] = TPdict[i] / (TPdict[i] + FNdict[i])
    F1Dict[i] = 2 / (1 / PrecisionDict[i] + 1 / RecallDict[i])
    SupportDict[i] = TPdict[i] + FNdict[i]

In [ ]:
columns = ['Precision', 'Recall', 'F1', 'Support']
metrics = {}

for i in zip(PrecisionDict.items(), RecallDict.items(), F1Dict.items(), SupportDict.items()):
    metrics[i[0][0]] = [i[0][1], i[1][1], i[2][1], i[3][1]]
    
metricsDf = pd.DataFrame().from_dict(metrics, orient='index')
metricsDf.columns = columns

In [ ]:
metricsDf

In [ ]:
metricsDf.to_excel('metrics3_24.xls', engine='xlsxwriter')

## [ошибка](https://colab.research.google.com/github/KevinMusgrave/pytorch-metric-learning/blob/master/examples/notebooks/TripletMarginLossMNIST.ipynb)
accuracies = accuracy_calculator.get_accuracy